## KITTI Dataset Tutorial

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
from matplotlib import gridspec
from matplotlib import pyplot as plt
import PIL.Image as pil

### Load and visualize stereo images
Note: These are stereo pair images from the same timestamp.

In [ ]:
stereo_left_image_path = "data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/image_02/data/0000000000.png"
stereo_right_image_path = "data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/image_03/data/0000000000.png"

stereo_left_image = pil.open(stereo_left_image_path).convert('RGB')
stereo_right_image = pil.open(stereo_right_image_path).convert('RGB')

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(stereo_left_image)
plt.show()

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(stereo_right_image)
plt.show()

In [ ]:
fig = plt.figure(figsize=(20, 10)) 
grid = gridspec.GridSpec(2, 1, height_ratios=[1, 1], wspace=0.0, hspace=0.0, figure=fig)

# Plot camera image.
ax = plt.subplot(grid[0])
ax.imshow(stereo_left_image)
ax.set_xticklabels([])
ax.set_yticklabels([])

# Plot segmentation image.
ax = plt.subplot(grid[1])
ax.imshow(stereo_right_image)
ax.set_xticklabels([])
ax.set_yticklabels([])

plt.show()

### Load and Visualize Lidar Data.
Note: Lidar is formatted as [N, 4] where N is the number of lidar points and 4 corresponds to x, y, z, lidar_intensity.

In [3]:
import kitti_utils

lidar_data_path = 'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/velodyne_points/data/0000000000.bin'
lidar_data = kitti_utils.load_lidar_points(lidar_data_path)

In [ ]:
import plotly.graph_objects as go
import plotly_utils

fig = go.Figure()
data = go.Scatter3d(x=lidar_data[:, 0], 
                    y=lidar_data[:, 1], 
                    z=lidar_data[:, 2], 
                    mode='markers',
                    marker=dict(size=1, color=lidar_data[:, 3], colorscale='Viridis'),
                    name='lidar')
fig.add_traces(data)

plotly_utils.setup_layout(fig)
fig.show()

### Loading IMU Data

In [ ]:
scene_path = 'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/'

imu_data = kitti_utils.get_imu_data(scene_path, 0)
imu_data

In [ ]:
imu_df = kitti_utils.get_imu_dataframe(scene_path)
imu_df

In [ ]:
import pytest
@pytest.fixture
def kitti_root_directory():
    return 'data/kitti_example'

@pytest.fixture
def kitti_dataset_index():
    test_data = {'path_name': ['data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/', 
                               'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/',
                                'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/'],
                 'index': ['0', '1', '2']}
    
    return pd.DataFrame(test_data, columns = ['path_name', 'index'])

def test_something(kitti_root_directory, kitti_dataset_index):
    dataset = KittiDataset(root_dir=kitti_root_directory, 
                           dataset_index=kitti_dataset_index, 
                           previous_frame=1,
                           next_frame=1)
    
data = dataset[0]
#print(dataset[0])
#print(dataset[0])
print(dataset[1]['nearby_frames'].keys())
print(dataset[2]['nearby_frames'])

In [4]:
from dataloader import KittiDataset
dataset = KittiDataset.init_from_config('configs/kitti_dataset.yml')
import pandas as pd
TEST_DATASET_INDEX = {'path_name': ['data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/', 
                                    'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/',
                                    'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/'],
                      'index': ['0', '1', '2']}
dataset_index = pd.DataFrame(TEST_DATASET_INDEX, columns = ['path_name', 'index'])
print(dataset_index['path_name'])

0    data/kitti_example/2011_09_26/2011_09_26_drive...
1    data/kitti_example/2011_09_26/2011_09_26_drive...
2    data/kitti_example/2011_09_26/2011_09_26_drive...
Name: path_name, dtype: object


In [5]:
from kitti_utils_test import kitti_root_directory, kitti_dataset_index

kitti_root_directory = 'data/kitti_example'
test_data = {'path_name': ['data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/', 
                           'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/',
                           'data/kitti_example/2011_09_26/2011_09_26_drive_0048_sync/',
                           'data/kitti_example/2011_09_26/2011_09_26_drive_0060_sync/', 
                           'data/kitti_example/2011_09_26/2011_09_26_drive_0060_sync/',
                           'data/kitti_example/2011_09_26/2011_09_26_drive_0060_sync/'],
             'index': ['0', '1', '2', '0', '1', '2']}
kitti_dataset_index = pd.DataFrame(test_data, columns = ['path_name', 'index'])

dataset = KittiDataset(root_dir=kitti_root_directory, 
                       dataset_index=kitti_dataset_index, 
                       previous_frames=2,
                       next_frames=2)

In [22]:
print([(k, bool(v)) for k, v in dataset[2]['nearby_frames'].items()])

[(-2, True), (-1, True), (1, True), (2, True)]


In [16]:
import os
os.listdir('data/kitti_example/2011_09_26')

['calib_velo_to_cam.txt',
 '.DS_Store',
 '2011_09_26_drive_0048_sync',
 'calib_imu_to_velo.txt',
 'calib_cam_to_cam.txt']

In [19]:
previous_frames = 2
all_frames = range(-previous_frames, 3)

In [20]:
print(all_frames)

range(-2, 3)


In [21]:
for num in all_frames:
    print(num)

-2
-1
0
1
2


In [5]:
from dataloader import KittiDataset
import pandas as pd

In [30]:
data = dataset[0]
print(data)

{'stereo_left_image': array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 11,  12,  14],
        [ 11,  12,  14],
        [ 11,  12,  14]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 12,  12,  14],
        [ 12,  12,  14],
        [ 11,  12,  14]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [ 12,  15,  15],
        [ 12,  14,  15],
        [ 11,  12,  15]],

       ...,

       [[ 10,  11,  11],
        [ 10,  10,  12],
        [ 10,  10,  13],
        ...,
        [ 10,  11,  14],
        [  9,  11,  14],
        [  8,   9,  13]],

       [[ 10,  11,  13],
        [ 10,  10,  12],
        [ 10,   9,  12],
        ...,
        [ 10,  11,  12],
        [  9,  11,  12],
        [  8,  10,  12]],

       [[ 11,  11,  13],
        [ 11,  11,  12],
        [ 10,  10,  12],
        ...,
        [  8,   9,  13],
        [  8,   9,  12],
  

In [32]:
print(data['nearby_frames'][1].keys())

dict_keys(['stereo_left_image', 'stereo_left_shape', 'stereo_left_capture_time_nsec', 'stereo_right_image', 'stereo_right_shape', 'stereo_right_capture_time_nsec'])
